# Graph

> graph functions

In [ ]:
#| default_exp graph

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:

#| export
import itertools, math, numpy as np, pandas as pd
import dgl, dgl.function as fn

import torch, torch.nn as nn, torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torchdiffeq

from typing import Callable, Union, Literal

def dgl_norm(g:dgl.DGLGraph, epoch=None) -> dgl.DGLGraph:
    '''
    Compute the diagonal normalization matrix D**-0.5 for a given graph G

    Parameters
    ----------
    g
        Graph of which to compute norm of and stores it in `ndata["norm"]`.
    
    epoch
        Defaults to `None`. If provided creates and stores norm in `ndata["epochs_norm"]`.
    
    Returns
    -------
    g
        Given graph updated with norm calculated.
    '''
    
    # Compute diagonal of normalization matrix D according to standard formula
    degs = g.in_degrees().float()
    norm = torch.pow(degs, -0.5)
    norm[torch.isinf(norm)] = 0

    # Add to dgl.Graph in order for the norm to be accessible at training time
    g.ndata['norm'] = norm.unsqueeze(1).to(g.device)
    
    # Keep track of norm over epochs
    if epoch is not None:
        if 'epoch_norms' not in g.ndata:            
            g.ndata['epoch_norms'] = {}
        g.ndata['epoch_norms'][epoch] = norm.unsqueeze(1).to(g.device)        
    return g

In [ ]:
#| export

def create_node_map(
    df:pd.DataFrame, u_key:str='from', v_key:str='to', dropna:bool=False
) -> dict:
    '''
    Converts a Pandas DataFrame adjaceny list into a dictionary like {from: to}

    Parameters
    ----------
    df
        The adjacency DataFrame
    
    u_key
        Starting node key in df

    v_key
        Ending node key in df

    Returns
    -------
    adjaceny
        dict of form {from:to}
    '''
    if dropna:
        df = df.dropna()
    values = np.unique(df[[u_key, v_key]].values.flatten())
    factorized = pd.factorize(values, sort=True)
    mapping = dict(zip(factorized[1], factorized[0]))
    return mapping

def get_edges_from_dataframe(
    df, u_key='from', v_key='to', 
    dropna:bool=False, node_map:dict=None
):
    if dropna:
        df = df.dropna()
    if node_map is None:
        node_map = create_node_map(df, u_key, v_key, dropna)
    u_nodes, v_nodes = df[[u_key, v_key]].applymap(lambda e : node_map[e]).values.T
    return u_nodes, v_nodes


def get_nodes_from_dataframe(
    df, u_key='from', v_key='to', 
    dropna:bool=False, node_map:dict=None
):
    if dropna:
        df = df.dropna()
    if node_map is None:
        node_map = create_node_map(df, u_key, v_key, dropna)
    all_nodes = np.unique(df[[u_key, v_key]].applymap(lambda e : node_map[e]).values.flatten())
    return all_nodes

def make_fully_connected_edges(nodes):
    return np.array(list(filter(
        lambda e: e[0] != e[1], # no self loops
        list(itertools.product(nodes, nodes))
    ))).T


def get_node_color(node, df_colors, split_at='_TF', lookup_key='module_id', color_key='color'):
    name = node
    if split_at is not None:
        name = name.split(split_at)[0]
    
    mask = df_colors[lookup_key].str.contains(name)
    color = 'grey' if df_colors[mask].empty else df_colors[mask].color.values[0]
    return color


def create_edge_map(graph):
    if 'dgl' in str(type(graph)):
        graph = graph.cpu().to_networkx()
    if 'networkx' not in str(type(graph)):
        raise TypeError('Graph is not a NetworkX graph type')
    return {(u, v): i for i, (u, v, *_) in enumerate(graph.edges)}

In [ ]:
#| export
def get_attn_edges_at_t(graph, attns, t, threshold=0.1):
    '''
    Parameters
    ----------
    graph
        a NetworkX graph type (e.g. MultiDiGraph)
    
    attns
        an numpy array with shape `(t, e)` where `t` is the number of time bins
        and `e` is the number of edges. 
    
    t
        the timebin of your model you want to get the edges of

    threshold
        the attention value you want to compare against

    Returns
    -------
    edge_list
        list of edges, `[(u, v), ...]` from the provided graph, if the value of the
        edge at time `t >= threshold`
    '''

    nodes = list(graph.nodes())
    is_directed = graph.is_directed()
    edge_list = []
    
    if len(attns.shape) == 1:
        attns = attns.reshape(1, -1)
        
    only_one_t = False
    if attns.shape[0] == 1:
        only_one_t = True
        t = 0
    
    for e_idx, (u_node, v_node) in enumerate(graph.edges()):
        e_val = attns[t, e_idx]
        
        u_idx, v_idx = list(map(nodes.index, (u_node, v_node)))
        if e_val >= threshold:
            edge_list.append((u_node, v_node))
            if not is_directed:
                edge_list.append((v_node, u_node))
    return edge_list

In [ ]:
#| export
def get_attn_mat_at_t(graph, attns, t, edge_map=None):
    '''
    Parameters
    ----------
    graph
        a NetworkX graph type (e.g. MultiDiGraph)
    
    attns
        an numpy array with shape `(t, e)` where `t` is the number of time bins
        and `e` is the number of edges. 
    
    t
        the timebin of your model you want to get the edges of

    edge_map
        the mapping of edge `(u, v)` to its index in `attns`

    Returns
    -------
    adj_mat
        the n x n adjacency matrix
    '''

    nodes = list(graph.nodes())
    is_directed = graph.is_directed()
    
    adj_mat = np.zeros((len(nodes), len(nodes)))
    for e_idx, (u_node, v_node) in enumerate(graph.edges()):
        
        if edge_map is not None:
            e_idx = edge_map[(u_node, v_node)]
            
        e_val = attns[t, e_idx]
        u_idx, v_idx = list(map(nodes.index, (u_node, v_node)))
        adj_mat.itemset((u_idx, v_idx), e_val)
        
        if not is_directed:
            adj_mat.itemset((v_idx, u_idx), e_val)
    
    return adj_mat

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()